Multi Instance Learning using PyTorch

In [22]:
# Load required libraries

import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from torchvision.models import resnet
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torch.utils.data.dataset import Dataset
from sklearn import model_selection as ms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from functools import partial, reduce
from torch.nn import functional as F
from tqdm.autonotebook import tqdm
from sklearn import metrics as mtx
from random import shuffle
import inspect
import time
from torch import nn, optim
import os
import time
import datetime
import copy
import re
import yaml
import uuid
import warnings
import random
import pickle


In [23]:
# Pretrain the model
def get_data(trainbatchsize, valbatchsize):
    # download the mnist dataset 
    data_mnist = MNIST(download=True, train=True, root=".").train_data.float()
    
    # Normalize and resize the images 
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((data_mnist.mean()/255,), (data_mnist.std()/255,))])

    # Load the training dataset
    train_dataset = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)
    
    # Load the validation dataset
    val_dataset = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_dataset, val_dataset


In [24]:
train_batch_size = 256
val_batch_size = 256

train_data, valid_data = get_data(train_batch_size, val_batch_size)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [25]:
# Define the model
class Mnist_resnet(ResNet):
    def __init__(self):
        super(Mnist_resnet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      
    # Forward propagation
    def forward(self, x):
        return torch.softmax(super(Mnist_resnet, self).forward(x), dim=-1)


In [26]:
# Helper function
def metric_calculation(metric, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric).args:
        return metric(true_y, pred_y, average="macro")
    else:
        return metric(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")        

In [30]:
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# mnist resnet model
model = Mnist_resnet().to(device)

# Parameter specification
epochs = 5

# Get training and validation datasets
train_dataset, val_dataset = get_data(train_batch_size, val_batch_size)

# Loss function
loss_function = nn.CrossEntropyLoss()         

# optimizer
optimizer = optim.Adadelta(model.parameters())

losses = []
train_batches = len(train_dataset)
val_batches = len(val_dataset)

# loop for each epoch 
for epoch in range(epochs):
    total_loss = 0

    progress = tqdm(enumerate(train_data), desc="Loss: ", total=train_batches)

   
    # train  model
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device) 
        model.zero_grad() 
     
        # Forward propagation
        op = model(X)          
        # Loss function           
        loss = loss_function(op, y)    
        # Back propagation   
        loss.backward() 
        # Optimization                      
        optimizer.step()                       
        current_loss = loss.item()
        total_loss += current_loss
        # Display the progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # Releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    # Validate the model
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    

    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_dataset):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X) 
            # Calculate loss                                     
            val_losses += loss_function(outputs, y)
            # Get Predictions
            predicted_classes = torch.max(outputs, 1)[1]         
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
            # calculate  various metrics for a batch
                acc.append(
                    metric_calculation(metric, y.cpu(), predicted_classes.cpu())
                )
          
    print(f" Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/train_batches}, Validation Loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    # store to plot the learning curve 
    losses.append(total_loss/train_batches) 
print(f" training time: {time.time()-start_ts}s")

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")



 Epoch 1/5, Training Loss: 1.6603912622370618, Validation Loss: 1.5458694696426392
	     precision: 0.9395
	        recall: 0.9240
	            F1: 0.9228
	      accuracy: 0.9270


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Epoch 2/5, Training Loss: 1.4778810673571647, Validation Loss: 1.5546873807907104
	     precision: 0.9413
	        recall: 0.9205
	            F1: 0.9191
	      accuracy: 0.9164



 Epoch 3/5, Training Loss: 1.4720364479308432, Validation Loss: 1.4740383625030518
	     precision: 0.9890
	        recall: 0.9888
	            F1: 0.9885
	      accuracy: 0.9891



 Epoch 4/5, Training Loss: 1.4694096306537061, Validation Loss: 1.479733943939209
	     precision: 0.9843
	        recall: 0.9838
	            F1: 0.9834
	      accuracy: 0.9837



 Epoch 5/5, Training Loss: 1.4673347868817919, Validation Loss: 1.4752329587936401
	     precision: 0.9885
	        recall: 0.9872
	            F1: 0.9873
	      accuracy: 0.9879
 training time: 872.5362763404846s


In [31]:
# Save the model 
torch.save(model.state_dict(), 'mnist_state.pt')


# Data Generation


In [32]:
# Load the train test data for data generation
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [33]:
x_train = x_train[:3000]
y_train = y_train[:3000]
x_test = x_test[:900]
y_test = y_test[:900]

In [34]:
# Change the datatype to Float 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalise the values 
x_train /= 255
x_test /= 255
print('Shape of x_train: {}'.format(x_train.shape))
print('Number of images in x_train: {}'.format(x_train.shape[0]))
print('Number of images in x_test: {}'.format(x_test.shape[0]))

Shape of x_train: (3000, 28, 28)
Number of images in x_train: 3000
Number of images in x_test: 900


Create tuple for train and test

In [35]:
# Get a sample x value
instance_index_label = [(i, y_train[i]) for i in range(x_train.shape[0])]
instance_index_label_test = [(i, y_test[i]) for i in range(x_test.shape[0])]

In [36]:
# Find the index if label is 1 in x_train labels  
find_index = [instance_index_label[i][0] for i in range(len(instance_index_label)) if instance_index_label[i][1]==1]
# Find the index if label is 1 in x_test labels 
find_index_test = [instance_index_label_test[i][0] for i in range(len(instance_index_label_test))
                   if instance_index_label_test[i][1]==1]

In [38]:
print('X_train Index:', instance_index_label[0][0])
print('X_train Label:', instance_index_label[0][1])       

X_train Index: 0
X_train Label: 5


In [39]:
import torch
from torchvision.models.resnet import ResNet, BasicBlock

# Define the resnet model
class Mnist_ResNet(ResNet):
    def __init__(self):
        super(Mnist_ResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(Mnist_ResNet, self).forward(x), dim=-1)

In [40]:
model = Mnist_ResNet()
# Load the pre-trained model
model.load_state_dict(torch.load('mnist_state.pt'))
body = nn.Sequential(*list(model.children()))
# Take the label values
model = body[:9]
# Evaluate the model
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [41]:
# Get the features
train_batch_size = 1
val_batch_size = 1
# Load the training and validation dataset 
train_dataset, val_dataset = get_data(train_batch_size, val_batch_size)
# Define the cross entropy loss 
loss_function = nn.CrossEntropyLoss()
# Optimization 
optimizer = optim.Adadelta(model.parameters())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [42]:
losses = []
batches = len(train_dataset)
val_batches = len(val_dataset)

Get features for train


In [43]:
# loop for every epoch
meta_table = dict()
feature_result = []


progress = tqdm(enumerate(train_dataset), desc="Loss: ", total=batches)

model.eval()

for i, data in progress:
    if i==3001:
        break
    X, y = data[0], data[1]
    # training for every loop
    model.zero_grad()
    op = model(X)
    feature_result.append(op.reshape(-1).tolist())
    meta_table[i] = op.reshape(-1).tolist()
    
feature_array = np.array(feature_result)
np.save('feature_full',feature_array )

In [44]:
# Load the features
feature_array = np.load('feature_full.npy', allow_pickle=True)

Generate features for test

In [47]:
meta_table = dict()
feature_result = []

progress = tqdm(enumerate(val_dataset), desc="Loss: ", total=batches)

model.eval()

for i, data in progress:
    if i==900:
        break
    X, y = data[0], data[1]
    model.zero_grad()
    op_test = model(X)
    feature_result.append(op_test.reshape(-1).tolist())
    meta_table[i] = op_test.reshape(-1).tolist()

feature_test_arr = np.array(feature_result)
# save 
np.save('feature_test_full',feature_test_arr )

In [48]:
# Load the validation features
feature_test_arr = np.load('feature_test_full.npy', allow_pickle=True)

Generate data for train

In [49]:
def generate_data(instance_index_label):
    
    bag_size = np.random.randint(3,7,size=len(instance_index_label)//5)
    data_cp = copy.copy(instance_index_label)
    np.random.shuffle(data_cp)
    bags = {}
    bags_per_instance_labels = {}
    bags_labels = {}
    for bag_ind, size in enumerate(bag_size):
        bags[bag_ind] = []
        bags_per_instance_labels[bag_ind] = []
        # Find the labels for every bag value 
        try:
            for _ in range(size):
                inst_ind, lbl = data_cp.pop()
                bags[bag_ind].append(inst_ind)
                bags_per_instance_labels[bag_ind].append(lbl)
            bags_labels[bag_ind] = bag_label_from_instance_labels(bags_per_instance_labels[bag_ind])
        except:
            break
    return bags, bags_labels

def bag_label_from_instance_labels(instance_labels):
    return int(any(((x==1) for x in instance_labels)))

In [50]:
bag_indices, bag_labels = generate_data(instance_index_label)
bag_features = {kk: torch.Tensor(feature_array[inds]) for kk, inds in bag_indices.items()}


In [51]:
# save
import pickle
pickle.dump(bag_indices, open( "bag_indices", "wb" ) )
pickle.dump(bag_labels, open( "bag_labels", "wb" ) )
pickle.dump(bag_features, open( "bag_features", "wb" ) )

In [52]:
# Load
import pickle
bag_indices = pickle.load( open( "bag_indices", "rb" ) )
bag_labels = pickle.load( open( "bag_labels", "rb" ) )
bag_features = pickle.load( open( "bag_features", "rb" ) )

In [55]:
# get the labels of the sample values 
bag_t_indices, bag_t_labels = generate_data(instance_index_label_test)


In [56]:
bag_t_features = {kk: torch.Tensor(feature_test_arr[inds]) for kk, inds in bag_t_indices.items()}


In [57]:
# Save bags for test data
pickle.dump(bag_t_indices, open( "bag_t_indices", "wb" ) )
pickle.dump(bag_t_labels, open( "bag_t_labels", "wb" ) )
pickle.dump(bag_t_features, open( "bag_t_features", "wb" ) )

In [58]:
bag_t_indices = pickle.load( open( "bag_t_indices", "rb" ) )
bag_t_labels = pickle.load( open( "bag_t_labels", "rb" ) )
bag_t_features = pickle.load( open( "bag_t_features", "rb" ) )

In [59]:
# Multiple Instance Learning

from torch.utils.data import Dataset
class Data_transformation(Dataset):

    def __init__(self, data, transform=None):
        self.transform = transform
        self.data = data
        
    def __getitem__(self, index):
        tensor = self.data[index][0]
        if self.transform is not None:
            tensor = self.transform(tensor)
        return (tensor, self.data[index][1])

    def __len__(self):
        return len(self.data)

In [60]:
train_data = [(bag_features[i],bag_labels[i]) for i in range(len(bag_features))]


In [63]:
bag_features[0]


tensor([[0.9466, 2.6843, 0.6233,  ..., 1.1060, 2.2033, 0.1852],
        [1.2178, 0.0893, 1.5062,  ..., 0.9497, 0.4214, 0.6974],
        [0.0279, 0.0153, 0.3912,  ..., 0.2437, 1.0758, 0.1476],
        [0.6781, 0.3141, 0.5197,  ..., 0.4018, 0.5470, 1.4038],
        [0.0374, 0.0178, 0.5753,  ..., 0.2824, 1.1689, 0.3425]])

As the bag has different sizes, in order to have the same shape (maximum 7) we will pad each tensor. We will check each instance and the shape of the tensor, then we will pad 7-n to the existing tensor. Here, n is total number of instances in the bag.

In [64]:
def tensor_padding(data, max_number_instance):
    new_data = []
    for bag_index in range(len(data)):
        tensor_size = len(data[bag_index][0])
        pad_size = max_number_instance - tensor_size
        p2d = (0,0, 0, pad_size)
        padded = nn.functional.pad(data[bag_index][0], p2d, 'constant', 0)
        new_data.append((padded, data[bag_index][1]))
    return new_data

In [65]:
max_num_instance = 7
padded_train = tensor_padding(train_data, max_num_instance)

In [68]:
test_data = [(bag_t_features[i],bag_t_labels[i]) for i in range(len(bag_t_features))]
padded_test = tensor_padding(test_data, max_num_instance)

In [69]:
def data_loaders(train_data, test_data, train_batch_size, val_batch_size):
    train_loader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(test_data, batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader

In [70]:
train_data_loader,valid_data_loader = data_loaders(padded_train, padded_test, 1, 1)


In [71]:
# Set the batch sizes
train_batch_size = 1
val_batch_size = 1

In [72]:
# Define a linear model 
class linear_model(torch.nn.Module):

    def __init__(self, n=7*512, n_out=1, dropout=0.2):
        super(linear_model, self).__init__()
        self.linear_1 = torch.nn.Linear(n, n_out)
        
    def forward(self, x):
        z = self.linear_1(x)
        y_pred = torch.sigmoid(z)
        return y_pred

In [73]:
# Define Neural Net model 
class NeuralNet(torch.nn.Module):

    def __init__(self, n=7*512, n_mid = 7168, n_out=1, dropout=0.2):
        super(NeuralNet, self).__init__()
        self.linear_1 = torch.nn.Linear(n, n_mid)
        self.linear_2 = torch.nn.Linear(n_mid, n_out)
        self.dropout = torch.nn.Dropout(dropout)
        self.non_linearity = torch.nn.LeakyReLU()
        
    def forward(self, x):
        z = self.linear_1(x)
        z = self.non_linearity(z)
        z = self.dropout(z)
        z = self.linear_2(z)
        y_pred = torch.sigmoid(z)
        return y_pred

In [74]:
class Noisy_And(torch.nn.Module):
    def __init__(self, a=10, dims=[0]):
        super(Noisy_And, self).__init__()
        self.a = a
        self.b = torch.nn.Parameter(torch.tensor(0.01))
        self.dims =dims
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        mean = torch.mean(x, self.dims, False)
        result = (self.sigmoid(self.a * (mean - self.b)) - self.sigmoid(-self.a * self.b)) / (self.sigmoid(self.a * (1 - self.b)) - self.sigmoid(-self.a * self.b))
        return result


class MIL_NeuralNet(torch.nn.Module):

    def __init__(self, n=7*512,  n_mid=7168, n_out=1, n_inst=None, dropout=0.1, noisy_a=4,agg = Noisy_And(a=4, dims=[0]),):
        super(MIL_NeuralNet, self).__init__()
        if agg is None:
            agg = Noisy_And(a=noisy_a, dims=[0])
        if n_inst is None:
            self.mdl_instance = agg
            n_inst = n
        else:
            self.mdl_instance = nn.Sequential(nn.Linear(n, n_inst), nn.LeakyReLU(),agg,)
        if n_mid == 0:
            self.mdl_bag = LogisticRegression(n_inst, n_out)
        else:
            self.mdl_bag = NeuralNet(n_inst, n_mid, n_out, dropout=dropout)

      

    def forward(self, bag_feature):
        y_pred = self.mdl_bag(bag_feature)
        return y_pred

In [79]:
def metric_calculation(metricfn, true_y, pred_y):
    # Multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metricfn).args:
        return metricfn(true_y, pred_y, average="macro")
    else:
        return metricfn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    # print the evaluation scores
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [81]:
import numpy as np
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr0 = 1e-4


model = MIL_NeuralNet().to(device)

epochs = 10
train_dataset, val_dataset = data_loaders(padded_train, padded_test, 1, 1)
loss_function = torch.nn.BCELoss(reduction='mean')


optimizer = optim.SGD(model.parameters(), lr=lr0, momentum=0.9)

losses = []
batches = len(train_dataset)
valbatches = len(val_dataset)

for epoch in range(epochs):
    total_loss = 0

    progress = tqdm(enumerate(train_dataset), desc="Loss: ", total=batches)


    model.train()
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        X = X.reshape([1,7*512])
        y = y.type(torch.cuda.FloatTensor)
        model.zero_grad() 
       
        op = model(X)                             
        loss = loss_function(op, y)              
        loss.backward()                              
        optimizer.step()                               

        current_loss = loss.item()
        total_loss += current_loss

        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_dataset):
            X, y = data[0].to(device), data[1].to(device)
            X = X.reshape([1,7*512])
            y = y.type(torch.cuda.FloatTensor)
            op = model(X)                         
            prediced_classes =op.detach().round()
            val_losses += loss_function(op, y)
            
           
            for acc, metric in zip((precision, recall, f1, accuracy), (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(metric_calculation(metric, y.cpu(), prediced_classes.cpu()))
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)                 
print(f"Training time: {time.time()-start_ts}s")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10, training loss: 0.7418684084465106, validation loss: 0.013402124866843224
	     precision: 0.0099
	        recall: 0.0099
	            F1: 0.0099
	      accuracy: 0.0099



Epoch 2/10, training loss: 0.6706850923225284, validation loss: 0.012501087971031666
	     precision: 0.0100
	        recall: 0.0100
	            F1: 0.0100
	      accuracy: 0.0100



Epoch 3/10, training loss: 0.6373887759322922, validation loss: 0.01330658420920372
	     precision: 0.0099
	        recall: 0.0099
	            F1: 0.0099
	      accuracy: 0.0099


...


Epoch 6/10, training loss: 0.5555258552605907, validation loss: 0.013318559154868126
	     precision: 0.0099
	        recall: 0.0099
	            F1: 0.0099
	      accuracy: 0.0099



Epoch 7/10, training loss: 0.5115367820424338, validation loss: 0.013328847475349903
	     precision: 0.0095
	        recall: 0.0095
	            F1: 0.0095
	      accuracy: 0.0095



Epoch 8/10, training loss: 0.4874965001953145, validation loss: 0.015031152404844761
	     precision: 0.0074
	        recall: 0.0074
	            F1: 0.0074
	      accuracy: 0.0074



Epoch 9/10, training loss: 0.45640801327923936, validation loss: 0.014501799829304218
	     precision: 0.0093
	        recall: 0.0093
	            F1: 0.0093
	      accuracy: 0.0093



Epoch 10/10, training loss: 0.42291156891733406, validation loss: 0.015001293271780014
	     precision: 0.0100
	        recall: 0.0100
	            F1: 0.0100
	      accuracy: 0.0100
Training time: 34.33377528190613s
